<a href="https://colab.research.google.com/github/nurmayadi/stemi-robotika/blob/master/Image_Classification_Fitra_Nurmayadi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dicoding Submission

##Fitra Nurmayadi

Proyek ini adalah untuk mengklasifikasikan jenis hewan, dan menjadikan model ML ke TFLite untuk dapat dipakai pada implementasi di mobile.

In [2]:
!pip install -q kaggle

In [7]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}"'.format(
      name=fn))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json"


In [8]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


#Import Libraries

In [6]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#Data Acquisition

menggunakan data animals10 dari kaggle

In [ ]:
!kaggle datasets download -d alessiocorrado99/animals10

In [ ]:
!unzip -qq animals10.zip -d "animals10"

In [ ]:
base_dir = "/content/animals10/raw-img"

In [ ]:
print(os.listdir(base_dir))

###Rename Folder

In [ ]:
from pathlib import Path
import shutil


folder_names = os.listdir(base_dir)

def rename_folder(current_name: str,
                  new_name: str,
                  path: Path) -> None:
    os.rename(os.path.join(path, current_name), os.path.join(path, new_name))

for folder in folder_names:
    current_folder = base_dir

    try:
        if folder == "cane":
            rename_folder(folder, "dog", current_folder)
        elif folder == "cavallo":
            rename_folder(folder, "horse", current_folder)
        elif folder == "elefante":
            rename_folder(folder, "elephant", current_folder)
        elif folder == "farfalla":
            rename_folder(folder, "butterfly", current_folder)
        elif folder == "gallina":
            rename_folder(folder, "chicken", current_folder)
        elif folder == "gatto":
            rename_folder(folder, "cat", current_folder)
        elif folder == "mucca":
            rename_folder(folder, "cow", current_folder)
        elif folder == "pecora":
            rename_folder(folder, "sheep", current_folder)
        elif folder == "ragno":
            rename_folder(folder, "spider", current_folder)
        elif folder == "scoiattolo":
            rename_folder(folder, "squirrel", current_folder)
    except Exception:
        shutil.rmtree(current_folder +"/"+ folder)
print("Folders renamed.")

In [ ]:
print(os.listdir(base_dir))

##Data Understanding

Cek jumlah gambar

In [ ]:
number_label = {}
total_files = 0
for i in os.listdir(base_dir):
    counting = len(os.listdir(os.path.join(base_dir, i)))
    number_label[i] = counting
    total_files += counting

print("Total Files : " + str(total_files))

Cek persebaran jumlah gambar setiap kelas data

In [1]:
plt.bar(number_label.keys(), number_label.values());
plt.title("Jumlah Gambar per Kelas");

NameError: ignored

Tampilkan sampel gambar untuk mengetahui ukurannya

In [ ]:
import matplotlib.image as mpimg

img_each_class = 1
img_samples = {}
classes = list(number_label.keys())


for c in classes:
    temp = os.listdir(os.path.join(base_dir, c))[:img_each_class]
    for item in temp:
        img_path = os.path.join(base_dir, c, item)
        img_samples[c] = img_path

for i in img_samples:
    fig = plt.gcf()
    img = mpimg.imread(img_samples[i])
    plt.title(i)
    plt.imshow(img)
    plt.show()

ukuran gambar bervariasi

###Split Folder: Train and Validation Folder

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
input_folder = base_dir

splitfolders.ratio(
    input_folder,
    output= "/content/animals10/dataset",
    seed=9,
    ratio=(0.8, 0.2),
    move=True,
    group_prefix=None
)
print("Moving files finished.")

os.listdir(input_folder)

for i in os.listdir(input_folder):
    if len(os.listdir(input_folder+"/"+ i)) == 0:
        os.rmdir(input_folder+"/"+ i)
print("Empty folders deleted.")

In [ ]:
train_dir = "/content/animals10/dataset/train"
valid_dir = "/content/animals10/dataset/val"

#Data Prepraration and Augmentation

In [ ]:
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = (IMAGE_SIZE[0],IMAGE_SIZE[0],3)
BATCH_SIZE = 32
SEED = 999

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    rescale = (1./255)
    )

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = (1./255)
    )

In [ ]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    class_mode='categorical',
    target_size=(IMAGE_SIZE[0],IMAGE_SIZE[0]),
    batch_size=BATCH_SIZE,
    seed=SEED
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    class_mode='categorical',
    target_size=(IMAGE_SIZE[0],IMAGE_SIZE[0]),
    batch_size=BATCH_SIZE,
    seed=SEED
)

#Callback Function

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')> 0.92 and logs.get('val_accuracy') > 0.92 ):
      print("\nTrain Accruracy and Validation Accuracy Was Reach > 92%!")
      self.model.stop_training = True
callbacks = myCallback()

#Accuracy Plot Function

In [ ]:
def acc_loss_plot(acc, val_acc, loss, val_loss):
  fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (19,7))
  sns.set_style("darkgrid")

  ax[0].plot(acc, 'o-',label = 'Training accuracy')
  ax[0].plot(val_acc, 'o-',label = 'Validation accuracy')
  ax[0].set_xlabel('Epochs')
  ax[0].set_ylabel('Accuracy')
  ax[0].set_title('Training Accuracy vs Epochs', fontsize = 17)
  ax[0].legend(loc='best')


  ax[1].plot(loss, 'o-',label = 'Training loss')
  ax[1].plot(val_loss, 'o-',label = 'Validation loss')
  ax[1].set_xlabel('Epochs')
  ax[1].set_ylabel('loss')
  ax[1].set_title('Loss vs. Epochs', fontsize = 17)
  ax[1].legend(loc='best')
  sns.set_style("darkgrid")

#Model Design



##1.CNN

In [ ]:
cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=IMAGE_SHAPE),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
cnn_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adamax(),
    metrics=['accuracy']
  )

In [ ]:
cnn_model.summary()

###CNN Train

In [ ]:
cnn_hist = cnn_model.fit(
    train_data,
    epochs=25,
    steps_per_epoch=50,
    validation_data=valid_data,
    callbacks=[callbacks]
    )

In [ ]:
acc = cnn_hist.history['accuracy']
val_acc = cnn_hist.history['val_accuracy']
loss = cnn_hist.history['loss']
val_loss = cnn_hist.history['val_loss']

acc_loss_plot(acc, val_acc, loss, val_loss)

##2.VGG16

In [ ]:
base_vgg_model = tf.keras.applications.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=IMAGE_SHAPE)

base_vgg_model.trainable = False

vgg_preprocess = tf.keras.applications.vgg16.preprocess_input
train_data.preprocessing_function = vgg_preprocess

In [ ]:
base_vgg_model.summary()

In [ ]:
vgg_model = tf.keras.models.Sequential([
    base_vgg_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
vgg_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adamax(),
    metrics=['accuracy']
  )

In [ ]:
vgg_model.summary()

###VGG16 Train

In [ ]:
vgg_hist = vgg_model.fit(
    train_data,
    epochs=32,
    validation_data=valid_data,
    validation_split=0.1,
    callbacks=[callbacks]
    )

In [ ]:
acc = vgg_hist.history['accuracy']
val_acc = vgg_hist.history['val_accuracy']
loss = vgg_hist.history['loss']
val_loss = vgg_hist.history['val_loss']

acc_loss_plot(acc, val_acc, loss, val_loss)

##3.EffecientNetV2B1

In [ ]:
base_env2_model = tf.keras.applications.EfficientNetV2B1(
    weights="imagenet",
    include_top=False,
    input_shape=IMAGE_SHAPE)

base_env2_model.trainable = False

env2_preprocess = tf.keras.applications.efficientnet_v2.preprocess_input
train_data.preprocessing_function = env2_preprocess

In [ ]:
base_env2_model.summary()

In [ ]:
env2_model = tf.keras.models.Sequential([
    base_env2_model,
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
env2_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
  )

In [ ]:
env2_model.summary()

###EffecientNetV2 Train

In [ ]:
env2_hist = env2_model.fit(
    train_data,
    epochs=20,
    steps_per_epoch=50,
    validation_data=valid_data,
    validation_split=0.1,
    callbacks=[callbacks]
    )

In [ ]:
acc = env2_hist.history['accuracy']
val_acc = env2_hist.history['val_accuracy']
loss = env2_hist.history['loss']
val_loss = env2_hist.history['val_loss']

acc_loss_plot(acc, val_acc, loss, val_loss)